<a href="https://colab.research.google.com/github/cdbm/airbnb-price-rediction-optimizer/blob/main/airbnb_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd

# Database pre processing

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/cdbm/airbnb-price-rediction-optimizer/main/AB_NYC_2019.csv")
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [4]:
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [5]:
df = df.drop('name', axis=1)
df = df.drop('host_name', axis=1)
df = df.drop('last_review', axis=1)

In [6]:
df = df[df.price <= 500]

In [7]:
# df.loc[df['price'] <= 50, 'ranges'] = '0_TO_50'
df.loc[df['price'] <= 100, 'ranges'] = '0_TO_100'
df.loc[(df['price'] > 100) & (df['price'] <= 200), 'ranges'] = '101_TO_200'
# df.loc[(df['price'] > 150) & (df['price'] <= 200), 'ranges'] = '151_TO_200'
df.loc[(df['price'] > 200) & (df['price'] <= 300), 'ranges'] = '201_TO_300'
#df.loc[(df['price'] > 250) & (df['price'] <= 300), 'ranges'] = '251_TO_300'
df.loc[(df['price'] > 300) & (df['price'] <= 400), 'ranges'] = '301_TO_400'
#df.loc[(df['price'] > 350) & (df['price'] <= 400), 'ranges'] = '351_TO_400'
df.loc[(df['price'] > 400) & (df['price'] <= 500), 'ranges'] = '401_TO_500'
#df.loc[(df['price'] > 450) & (df['price'] <= 500), 'ranges'] = '551_TO_500'

In [8]:
df['neighbourhood_group'] = df['neighbourhood_group'].astype('category')
df['neighbourhood'] = df['neighbourhood'].astype('category')
df['room_type'] = df['room_type'].astype('category')

In [9]:
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [10]:
df['ranges'] = df['ranges'].astype('category')

In [11]:
df['reviews_per_month'].fillna(0, inplace=True) 

In [12]:
df = df.drop('price', axis=1)


In [13]:
df.head()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,ranges
0,2539,2787,1,107,40.64749,-73.97237,1,1,9,0.21,6,365,101_TO_200
1,2595,2845,2,126,40.75362,-73.98377,0,1,45,0.38,2,355,201_TO_300
2,3647,4632,2,93,40.80902,-73.94190,1,3,0,0.00,1,365,101_TO_200
3,3831,4869,1,41,40.68514,-73.95976,0,1,270,4.64,1,194,0_TO_100
4,5022,7192,2,61,40.79851,-73.94399,0,10,9,0.10,1,0,0_TO_100


In [14]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:10000]
df.describe()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.000000e+04,1.000000e+04,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.900122e+07,6.755997e+07,1.666800,106.86950,40.728859,-73.951714,0.505900,7.036400,23.322300,1.094102,6.704600,112.458900
std,1.091262e+07,7.885536e+07,0.741912,68.77964,0.054803,0.046412,0.546987,18.900423,43.957303,1.558275,30.871324,131.709418
min,5.022000e+03,2.438000e+03,0.000000,0.00000,40.507080,-74.242850,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,9.554414e+06,7.695235e+06,1.000000,51.00000,40.689777,-73.982340,0.000000,1.000000,1.000000,0.050000,1.000000,0.000000
50%,1.970134e+07,3.077882e+07,2.000000,93.00000,40.722860,-73.954895,0.000000,3.000000,5.000000,0.390000,1.000000,43.000000
75%,2.896087e+07,1.074344e+08,2.000000,180.00000,40.763732,-73.936187,1.000000,5.000000,24.000000,1.610000,2.000000,228.000000
max,3.648724e+07,2.742733e+08,4.000000,218.00000,40.911690,-73.716900,2.000000,500.000000,540.000000,20.940000,327.000000,365.000000


In [15]:
df

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,ranges
0,35212471,202805108,2,94,40.76028,-73.99665,0,3,3,3.00,1,102,301_TO_400
1,23810398,178696800,1,51,40.67825,-73.94748,1,1,45,2.83,1,28,0_TO_100
2,34686718,248555214,3,4,40.76417,-73.92898,1,2,4,4.00,2,8,0_TO_100
3,16353485,32545798,1,213,40.71443,-73.96154,1,1,4,0.13,5,0,0_TO_100
4,31760509,219505617,2,93,40.80351,-73.95822,1,1,18,3.62,1,92,0_TO_100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,22204187,18011536,2,35,40.71799,-73.99640,0,2,0,0.00,1,0,101_TO_200
9996,21921707,100424005,1,28,40.69386,-73.91897,1,5,9,0.46,1,0,0_TO_100
9997,177421,848748,1,90,40.72315,-73.95226,0,2,20,0.21,2,127,401_TO_500
9998,10827567,31411094,1,51,40.67250,-73.93981,0,3,128,3.10,1,342,0_TO_100


In [16]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [17]:
  features = df.drop('ranges',axis=1)
  labels = df['ranges']

In [18]:
feature_train, feature_test, label_train, label_test = train_test_split(features, labels, test_size=0.2, random_state=1)

In [19]:
sc_X = StandardScaler()
feature_train=sc_X.fit_transform(feature_train)
feature_test=sc_X.fit_transform(feature_test)

# Genetic Algorithm

In [156]:
import random

In [157]:
POPULATION_SIZE = 30
GENERATIONS = 100
CHILDREN_RATE = 3

In [158]:
activation = ['identity', 'logistic', 'tanh', 'relu']
solver = ['lbfgs','sgd', 'adam']
learning_rate = ['constant', 'invscaling', 'adaptive']

In [159]:
def generate_individue():
  individue = {
      'activation':  activation[random.randint(0, 3)],
      'solver': solver[random.randint(0,2)],
      'learning_rate': learning_rate[random.randint(0,2)],
      'hidden_layer_sizes' : (random.randint(100, 200), random.randint(50, 100) ,random.randint(10, 50)),
      'max_iter' : random.randint(100, 250),
      'fitness': 0  
  }

  individue['fitness'] = calculate_fitness(individue)
  return individue

In [160]:
def sortByFitness(e):
  return e['fitness']

In [161]:
def initiate_population(population_size):
  population = [[]] * population_size
  population = [generate_individue() for i in range(0, population_size)]
  population.sort(key=sortByFitness, reverse=True)
  return population

In [162]:
def choose_parents(population):
  father = random.randint(0, POPULATION_SIZE - 1)
  mother = random.randint(0, POPULATION_SIZE - 1)
  return [population[father], population[mother]]

In [163]:
def choose_candidates(population):
  candidatesIndex = []
  for x in range(0, round(POPULATION_SIZE * 0.3) ):
    index = random.randint(0, POPULATION_SIZE - 1)
    while index in candidatesIndex:
      index = random.randint(0, POPULATION_SIZE - 1)
    candidatesIndex.append(index)
    
  candidates = []
  for x in candidatesIndex:
    candidates.append(population[x])
  return candidates   

def choose_parents_by_roullete(population):
  candidates = choose_candidates(population)
  probabilities = [];
  sumOfFitness = 0;
  previousProbabilities = 0;
  parents = [];

  for i in candidates:
    sumOfFitness = sumOfFitness + i['fitness']

  candidates.sort(key=sortByFitness)

 
  for cand in candidates:
    prob = previousProbabilities + (cand['fitness'] / sumOfFitness);
    previousProbabilities = prob;
    probabilities.append(prob);
  
  while len(parents)  < 2:
    number = random.randint(1, 10) /10
    found = False
    for y in range(0, len(probabilities)):
      if (number <= probabilities[y]) & (not found):
        parents.append(candidates[y])
        found = True
  
  return parents

In [164]:
def mutate_activation(individue):
  new_attribute = activation[random.randint(0,3)] 
  while new_attribute == individue['activation']:
    new_attribute = activation[random.randint(0,3)] 
  individue['activation'] = new_attribute
  return individue

def mutate_learning_rate(individue):
  new_attribute = learning_rate[random.randint(0,2)] 
  while new_attribute == individue['learning_rate']:
    new_attribute = learning_rate[random.randint(0,2)] 
  individue['learning_rate'] = new_attribute
  return individue

def mutate_solver(individue):
  new_attribute = solver[random.randint(0,2)] 
  while new_attribute == individue['solver']:
    new_attribute = solver[random.randint(0,2)] 
  individue['solver'] = new_attribute
  return individue

def mutate(individue):
  mutations = [mutate_activation, mutate_learning_rate, mutate_solver]
  prob = random.randint(0, 10)
  if prob >= 6:
    mutation1 = random.randint(0,2)
    mutation2 = random.randint(0,2)

    individue = mutations[mutation1](individue)
    individue = mutations[mutation2](individue)

  return individue

In [165]:
def make_children(population):
  parents = choose_parents_by_roullete(population)
  
  for x in range(0, 2):
    child = {'activation': parents[random.randint(0,1)]['activation'],
            'hidden_layer_sizes': (parents[random.randint(0,1)]['hidden_layer_sizes'][0], parents[random.randint(0,1)]['hidden_layer_sizes'][1], parents[random.randint(0,1)]['hidden_layer_sizes'][2]),
            'learning_rate': parents[random.randint(0,1)]['learning_rate'],
            'max_iter': parents[random.randint(0,1)]['max_iter'],
            'solver': parents[random.randint(0,1)]['solver'],
            'fitness': 0}
  
    child = mutate(child)
    child['fitness'] = calculate_fitness(child)
    population.append(child)
  
  population.sort(key=sortByFitness, reverse=True)
  return population[:POPULATION_SIZE]

In [166]:
def calculate_fitness(individue):
  model_mlp = MLPClassifier(activation=individue['activation'],
                            hidden_layer_sizes=individue['hidden_layer_sizes'],
                            learning_rate=individue['learning_rate'],
                            max_iter=individue['max_iter'],
                            solver=individue['solver'])
  model_mlp.fit(feature_train, label_train) 

  prediction = model_mlp.predict(feature_test) 
  return accuracy_score(label_test, prediction)
  

In [167]:
parents

[{'activation': 'logistic',
  'fitness': 0.702,
  'hidden_layer_sizes': (139, 45),
  'learning_rate': 'adaptive',
  'max_iter': 229,
  'solver': 'adam'}]

Main


In [168]:
population = initiate_population(POPULATION_SIZE)
population.sort(key=sortByFitness, reverse=True)
print(population)
for x in range(0, GENERATIONS):
  print("generation: ")
  print(x)
  population = make_children(population)
  print(population)

print("Better was: ") 
print(population[0])
print("With a accuracy of: ")
print(population[0]['fitness'])

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (217) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.

[{'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (187, 80, 24), 'max_iter': 103, 'fitness': 0.704}, {'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (147, 80, 32), 'max_iter': 176, 'fitness': 0.703}, {'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (149, 59, 30), 'max_iter': 175, 'fitness': 0.7015}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (192, 82, 27), 'max_iter': 116, 'fitness': 0.7}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (134, 59, 44), 'max_iter': 155, 'fitness': 0.6995}, {'activation': 'logistic', 'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (168, 50, 20), 'max_iter': 156, 'fitness': 0.699}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (115, 91, 46), 'max_iter': 114, '

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (103) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (167) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[{'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (187, 80, 24), 'max_iter': 103, 'fitness': 0.704}, {'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (147, 80, 32), 'max_iter': 176, 'fitness': 0.703}, {'activation': 'tanh', 'hidden_layer_sizes': (187, 80, 48), 'learning_rate': 'adaptive', 'max_iter': 167, 'solver': 'sgd', 'fitness': 0.702}, {'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (149, 59, 30), 'max_iter': 175, 'fitness': 0.7015}, {'activation': 'tanh', 'hidden_layer_sizes': (187, 80, 48), 'learning_rate': 'adaptive', 'max_iter': 103, 'solver': 'sgd', 'fitness': 0.7005}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (192, 82, 27), 'max_iter': 116, 'fitness': 0.7}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (134, 59, 44), 'max_iter': 155, 'fitness': 0

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

[{'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (187, 80, 24), 'max_iter': 103, 'fitness': 0.704}, {'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (147, 80, 32), 'max_iter': 176, 'fitness': 0.703}, {'activation': 'tanh', 'hidden_layer_sizes': (187, 80, 48), 'learning_rate': 'adaptive', 'max_iter': 167, 'solver': 'sgd', 'fitness': 0.702}, {'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (149, 59, 30), 'max_iter': 175, 'fitness': 0.7015}, {'activation': 'tanh', 'hidden_layer_sizes': (187, 80, 48), 'learning_rate': 'adaptive', 'max_iter': 103, 'solver': 'sgd', 'fitness': 0.7005}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (192, 82, 27), 'max_iter': 116, 'fitness': 0.7}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (134, 59, 44), 'max_iter': 155, 'fitness': 0

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (103) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (151) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[{'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (187, 80, 24), 'max_iter': 103, 'fitness': 0.704}, {'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (147, 80, 32), 'max_iter': 176, 'fitness': 0.703}, {'activation': 'tanh', 'hidden_layer_sizes': (187, 80, 48), 'learning_rate': 'adaptive', 'max_iter': 167, 'solver': 'sgd', 'fitness': 0.702}, {'activation': 'tanh', 'hidden_layer_sizes': (187, 100, 24), 'learning_rate': 'adaptive', 'max_iter': 103, 'solver': 'sgd', 'fitness': 0.702}, {'activation': 'tanh', 'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (149, 59, 30), 'max_iter': 175, 'fitness': 0.7015}, {'activation': 'tanh', 'hidden_layer_sizes': (187, 80, 48), 'learning_rate': 'adaptive', 'max_iter': 103, 'solver': 'sgd', 'fitness': 0.7005}, {'activation': 'logistic', 'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (192, 82, 27), 'max_iter': 116, 'fitness': 0.7}

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: ignored